#### ⏰ Até 15/02 ⏰ 1ª FASE - EXTRAÇÃO DOS DADOS DA ESCALA DE SERVIÇO 

In [ ]:
from docx import Document
import pandas as pd

# função que abre e extrai todas as tabelas de um docx
def extrairTabelas(caminhoDocx):
    todasTabelas = []

    doc = Document(caminhoDocx)
    
    for tabela_idx, tabela in enumerate(doc.tables):
        tabelaAtual = []
        
        for row_idx, row in enumerate(tabela.rows):
            cells = [cell.text.strip() for cell in row.cells]
            tabelaAtual.append(cells)

        todasTabelas.append(tabelaAtual)

    return todasTabelas

# coloca o nome do docx
tabelas = extrairTabelas('17.02.2024 ESCALA DE SERVIÇO - SÁBADO.docx')

# nomes das equipes de serviço para referenciar os dfs
nomesEquipes = ['dias','cpu','sede','liberdade','alemanha','gtm']

# cria o dicionário equipe-tabela a ser preenchido
dfEquipe = {}

# criação dos DataFrames a partir da extração em docx
for idx, tabela in enumerate(tabelas):
    if idx >= len(nomesEquipes):
        novaEquipe = tabela[0][0][:15].lower()
        nomesEquipes.append(novaEquipe)
    dfEquipe[nomesEquipes[idx]] = pd.DataFrame(tabela)
    display(dfEquipe[nomesEquipes[idx]])

#### ⏰ Até 20/02 ⏰ 2ª FASE - EXTRAÇÃO DOS DADOS DO CHECKLIST 

In [ ]:
import gspread

# armazenar JSON em local seguro após a fase de teste. Credenciais de Acesso por Conta de Serviço
gc = gspread.service_account('./service_account.json')

spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1k6Ugpr-XRiWQEFeNPEkGSODZW2vfx1m1r3GCNk8xnQo')

colunasRelatorio = ['DATA/HORA CHECKLIST', 'NOME', 'ID', 'VTR/MT','TIPO']

planilhaMotos = spreadsheet.get_worksheet(0)
dadosMotos = pd.DataFrame(planilhaMotos.get_all_values()[-50:])
dadosMotos = dadosMotos[[0,2,3,4,5]]
dadosMotos.columns = colunasRelatorio
display(dadosMotos)

planilhaCarros = spreadsheet.get_worksheet(1)
dadosCarros = pd.DataFrame(planilhaCarros.get_all_values()[-50:])
dadosCarros = dadosCarros[[0,2,3,4,5]]
dadosCarros.columns = colunasRelatorio
display(dadosCarros)

In [ ]:
inicioIntervalo = '17/02/2024 07:00:00'
fimIntervalo = '18/02/2024 07:00:00'

dadosMotosFiltrados = dadosMotos[(dadosMotos['DATA/HORA CHECKLIST'] >= inicioIntervalo) & (dadosMotos['DATA/HORA CHECKLIST'] <= fimIntervalo)]
display(dadosMotosFiltrados)

dadosCarrosFiltrados = dadosCarros[(dadosCarros['DATA/HORA CHECKLIST'] >= inicioIntervalo) & (dadosCarros['DATA/HORA CHECKLIST'] <= fimIntervalo)]
display(dadosCarrosFiltrados)

#### ⏰ Até 25/02 ⏰ 3ª FASE - COMPARAÇÃO DADOS DA ESCALA E CHECKLIST EXECUTADOS 

In [ ]:
dfRelatorio = pd.DataFrame(columns = colunasRelatorio)

def exibirDados(valor, colunaId):
    global dfRelatorio
    
    if valor in dadosCarrosFiltrados['ID'].values:
        todaLinhaDado = dadosCarrosFiltrados.loc[dadosCarrosFiltrados['ID'] == valor]
        equipeColuna = pd.DataFrame({'EQUIPE': [equipeAtual[colunaId][0].split('\n')[0] + ' ' + equipeAtual[colunaId][1]]})
        todaLinhaDado = todaLinhaDado.assign(EQUIPE=equipeColuna['EQUIPE'].iloc[0])
        dfRelatorio = pd.concat([dfRelatorio, todaLinhaDado], ignore_index=True)
        
    elif valor in dadosMotosFiltrados['ID'].values:
        todaLinhaDado = dadosMotosFiltrados.loc[dadosMotosFiltrados['ID'] == valor]
        equipeColuna = pd.DataFrame({'EQUIPE': [equipeAtual[colunaId][0].split('\n')[0] + ' ' + equipeAtual[colunaId][1]]})
        todaLinhaDado = todaLinhaDado.assign(EQUIPE=equipeColuna['EQUIPE'].iloc[0])
        dfRelatorio = pd.concat([dfRelatorio, todaLinhaDado], ignore_index=True)
       
    else:
        pgNome = equipeAtual.loc[equipeAtual[colunaId] == valor][colunaId - 2].iloc[0] + ' ' + equipeAtual.loc[equipeAtual[colunaId] == valor][colunaId - 1].iloc[0]
        guarnicao = equipeAtual[colunaId][0].split('\n')[0] + ' ' + equipeAtual[colunaId][1] 
        dfRelatorio = pd.concat([dfRelatorio, pd.DataFrame({'DATA/HORA CHECKLIST': '-','NOME': [pgNome], 'ID': [valor], 'VTR/MT':'-', 'TIPO':'-', 'EQUIPE': [guarnicao] })], ignore_index = True)

        
def processarEquipe(equipeAtual, colunaId):
    for valor in equipeAtual[colunaId][3:]:
        exibirDados(valor, colunaId)



for i in range(len(nomesEquipes)):
    equipeAtual = dfEquipe[nomesEquipes[i]]
    if nomesEquipes[i] not in ['dias', 'expediente admi', 'sede']:
        processarEquipe(equipeAtual, 3)

        if equipeAtual[3][1] == equipeAtual[8][1]:
            processarEquipe(equipeAtual, 8)
                               
        else:
            processarEquipe(equipeAtual, 8)

display(dfRelatorio)
           

#### ⏰ Até 1ª/03 ⏰ 4ª FASE - RELATÓRIO AO CHEFE DA 4ª SEÇÃO 

In [ ]:
for equipeValor in dfRelatorio['EQUIPE'].unique():
    cargaLoop = []
    for i, linha in dfRelatorio[dfRelatorio['EQUIPE'] == equipeValor].iterrows():
        if linha['VTR/MT'] == '-':
            cargaLoop.append('Nada')
        else:
            cargaLoop.append(linha['VTR/MT'])
            cargaLoop.append(linha['TIPO'])
    display(cargaLoop)
    print(f'Relatório da {equipeValor}')

        